# 21 layers, pre-trained weight

In [1]:
import tensorflow as tf
import numpy as np
import time
import os
import re

from PIL import Image as PI

# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

In [2]:
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess_ :
    saver = tf.train.import_meta_graph('D:/Backup_data/jongkeun/paper/190524/save_dir-494.meta')
    saver.restore(sess_, tf.train.latest_checkpoint('D:/Backup_data/jongkeun/paper/190524/'))
    
    graph = tf.get_default_graph()
    
#     print(graph.get_tensor_by_name('layer0_conv2d_1/kernel:0'))
    weight0_1, weight0_2 = sess_.run([graph.get_tensor_by_name('layer0_conv2d_1/kernel:0'), 
                                      graph.get_tensor_by_name('layer0_conv2d_2/kernel:0')])
    weight1, weight1_1, weight1_2 = sess_.run([graph.get_tensor_by_name('layer1_conv2d/kernel:0'), 
                                               graph.get_tensor_by_name('layer1_conv2d_1/kernel:0'),
                                               graph.get_tensor_by_name('layer1_conv2d_2/kernel:0')])
    weight2, weight2_1, weight2_2 = sess_.run([graph.get_tensor_by_name('layer2_conv2d/kernel:0'), 
                                               graph.get_tensor_by_name('layer2_conv2d_1/kernel:0'),
                                               graph.get_tensor_by_name('layer2_conv2d_2/kernel:0')])
    weight3, weight3_1, weight3_2 = sess_.run([graph.get_tensor_by_name('layer3_conv2d/kernel:0'), 
                                               graph.get_tensor_by_name('layer3_conv2d_1/kernel:0'),
                                               graph.get_tensor_by_name('layer3_conv2d_2/kernel:0')])
    
    
#     weight_ = weight[:,:,:,:]
#     print(weight_)

print("++++++++++++++++++ DONE ++++++++++++++++++")

W1111 20:42:35.582975  5752 deprecation.py:323] From C:\Users\whdrm\Anaconda3\lib\site-packages\tensorflow\python\training\saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


++++++++++++++++++ DONE ++++++++++++++++++


In [3]:
training_epochs = 500
batch_size = 20
image_resize = 128
image_width = image_resize
image_height = image_resize
initial_rate = 1e-5
label_len = 3

home_dir = "D:/Backup_data/jongkeun"

image_file = "/dir_images/190819_newTraining/"
test_image_file = "/dir_images/190309_new_test/"

label_file = "/csv/190819_newTraining.csv"
test_label_file = "/csv/190309_new_test.csv"

In [4]:
label_dir = home_dir + label_file
test_label_dir = home_dir + test_label_file

# label_dir = os.getcwd() + "/csv/181126.csv"
# test_label_dir = os.getcwd() + "/csv/181126_test.csv"

f_label = open(label_dir, 'r')
f_test_label = open(test_label_dir, 'r')

label_data = f_label.readlines()
test_label_data = f_test_label.readlines()

f_label.close()
f_test_label.close()

label_array = np.array(np.reshape(label_data, [-1, label_len]), dtype=np.int32)
test_label_array = np.array(np.reshape(test_label_data, [-1, label_len]), dtype=np.int32)

print(len(label_array))
print(len(test_label_array))

9182
1018


In [5]:
image_dir = home_dir + image_file
test_image_dir = home_dir + test_image_file

# image_dir = os.getcwd() + "/dir_images/181126/"
# test_image_dir = os.getcwd() + "/dir_images/181126_test/"

image_list = os.listdir(image_dir)
test_image_list = os.listdir(test_image_dir)

image_list = sorted(image_list, key=lambda x: (int(re.sub('\D','',x)),x))
test_image_list = sorted(test_image_list, key=lambda y: (int(re.sub('\D','',y)),y))

image_array = []
test_image_array = []

for i in range(len(image_list)) :
    image_list[i] = image_dir + image_list[i]
    image_array.append(np.array(PI.open(image_list[i]).resize((image_resize, image_resize)).convert('L')))

for j in range(len(test_image_list)) :
    test_image_list[j] = test_image_dir + test_image_list[j]
    test_image_array.append(np.array(PI.open(test_image_list[j]).resize((image_resize, image_resize)).convert('L')))

image_array = np.reshape(image_array, [-1, image_width*image_height])
test_image_array = np.reshape(test_image_array, [-1, image_width*image_height])

image_array = np.array(np.reshape(image_array, [-1, image_width, image_height, 1]), dtype=np.float32)
test_image_array = np.array(np.reshape(test_image_array, [-1, image_width, image_height, 1]), dtype=np.float32)

print(len(image_array))
print(len(test_image_array))

9182
1018


In [6]:
x = tf.placeholder(dtype=tf.float32, shape=[None, image_width, image_height, 1], name='input_image')
y_ = tf.placeholder(dtype=tf.float32, shape=[None, label_len], name='input_label')
learning_rate = tf.placeholder(dtype=tf.float32, name='learning_rate')

# W_pre_trained = tf.placeholder(dtype=tf.float32, shape=[3,3,8,128], name='pre_trained_weight')

is_training = tf.placeholder(dtype=tf.bool)

In [7]:
W_h0_1 = tf.placeholder(dtype=tf.float32, shape=[3,3,1,8], name='W_h0_1')
W_h0_2 = tf.placeholder(dtype=tf.float32, shape=[3,3,8,128], name='W_h0_2')

W_h1 = tf.placeholder(dtype=tf.float32, shape=[3,3,128,256], name='W_h1')
W_h1_1 = tf.placeholder(dtype=tf.float32, shape=[3,3,256, 128], name='W_h1_1')
W_h1_2 = tf.placeholder(dtype=tf.float32, shape=[3,3,128,256], name='W_h1_2')

W_h2 = tf.placeholder(dtype=tf.float32, shape=[3,3,256,512], name='W_h2')
W_h2_1 = tf.placeholder(dtype=tf.float32, shape=[3,3,512,256], name='W_h2_1')
W_h2_2 = tf.placeholder(dtype=tf.float32, shape=[3,3,256,512], name='W_h2_2')

W_h3 = tf.placeholder(dtype=tf.float32, shape=[3,3,512,512], name='W_h3')
W_h3_1 = tf.placeholder(dtype=tf.float32, shape=[3,3,512,1024], name='W_h3_1')
W_h3_2 = tf.placeholder(dtype=tf.float32, shape=[3,3,1024,512], name='W_h3_2')

In [8]:
def pooling(input_, dropout, is_training) :
    pooling = tf.layers.max_pooling2d(input_, [2, 2], [2, 2], padding='SAME')
    pooling = tf.layers.dropout(pooling, dropout, is_training)
    
    return pooling

def learning_layer(input_data, size, dropout, is_training) :
    layer = tf.layers.conv2d(input_data, size, [3, 3], padding='SAME')
    layer = tf.layers.max_pooling2d(layer, [2, 2], [2, 2], padding='SAME')
    layer = tf.layers.dropout(layer, dropout, is_training)
    
    return layer

def convolutional(input_, weight_) :
    conv_ = tf.nn.conv2d(input_, weight_, strides=[1,1,1,1], padding='SAME')
    hidden_ = tf.nn.relu(conv_)
    
    return hidden_

with tf.device('/gpu:0') :
    layer0_1 = convolutional(x, W_h0_1)
    layer0_2 = convolutional(layer0_1, W_h0_2)
    layer1 = convolutional(layer0_2, W_h1)
    layer1_ = pooling(layer1, dropout=0.7, is_training=is_training)
    
    layer1_1 = convolutional(layer1_, W_h1_1)
    layer1_2 = convolutional(layer1_1, W_h1_2)
    layer2 = convolutional(layer1_2, W_h2)
    layer2_ = pooling(layer2, dropout=0.7, is_training=is_training)
    
    layer2_1 = convolutional(layer2_, W_h2_1)
    layer2_2 = convolutional(layer2_1, W_h2_2)
    layer2_3 = convolutional(layer2_2, W_h3)
    layer2_4 = convolutional(layer2_3, W_h3_1)
    layer2_5 = convolutional(layer2_4, W_h3_2)
    layer3 = learning_layer(input_data=layer2_5, size=1024, dropout=0.7, is_training=is_training)
    
    layer3_1 = tf.layers.conv2d(layer3, 128, [3,3], padding='SAME')
    layer3_2 = tf.layers.conv2d(layer3_1, 512, [3,3], padding='SAME')
    layer3_3 = tf.layers.conv2d(layer3_2, 256, [3,3], padding='SAME')
    layer3_4 = tf.layers.conv2d(layer3_3, 1024, [3,3], padding='SAME')
    layer3_5 = tf.layers.conv2d(layer3_4, 128, [3,3], padding='SAME')
    layer3_6 = tf.layers.conv2d(layer3_5, 512, [3,3], padding='SAME')
    layer3_7 = tf.layers.conv2d(layer3_6, 256, [3,3], padding='SAME')
    layer3_8 = tf.layers.conv2d(layer3_7, 1024, [3,3], padding='SAME')
    layer4 = learning_layer(input_data=layer3_8, size=1024, dropout=0.7, is_training=is_training)

W1111 20:46:18.505731  5752 deprecation.py:323] From <ipython-input-8-23d39681c4d3>:2: max_pooling2d (from tensorflow.python.layers.pooling) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
W1111 20:46:18.633391  5752 deprecation.py:323] From <ipython-input-8-23d39681c4d3>:3: dropout (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dropout instead.
W1111 20:46:18.701236  5752 deprecation.py:323] From <ipython-input-8-23d39681c4d3>:8: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W1111 20:46:18.703224  5752 deprecation.py:506] From C:\Users\whdrm\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecate

In [9]:
fully_layer = tf.contrib.layers.flatten(layer4)
fully_layer = tf.layers.dense(fully_layer, 1024, activation=tf.nn.relu, name='fully_connected_layer')
fully_layer = tf.layers.dropout(fully_layer, 0.5, is_training, name='fully_connected_dropout')

fully_layer2 = tf.layers.dense(fully_layer, 512, activation=tf.nn.relu, name='fully_connected_layer2')
fully_layer2 = tf.layers.dropout(fully_layer2, 0.5, is_training, name='fully_connected_dropout2')

W1111 20:46:22.363440  5752 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W1111 20:46:22.364433  5752 deprecation.py:323] From C:\Users\whdrm\Anaconda3\lib\site-packages\tensorflow\contrib\layers\python\layers\layers.py:1634: flatten (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.flatten instead.
W1111 20:46:22.591835  5752 deprecation.py:323] From <ipython-input-9-f9981dd3a3a2>:2: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.


In [10]:
logits = tf.layers.dense(fully_layer2, label_len, activation=None)

with tf.name_scope('pred') : 
    pred = tf.nn.softmax(logits)
with tf.name_scope('loss') :
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_))
    
with tf.device('/gpu:0') :
    with tf.name_scope('train') :
            train = tf.train.AdamOptimizer(learning_rate).minimize(loss)

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y_, 1))
acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

label_output = tf.argmax(pred, 1)
label_input = tf.argmax(y_, 1)

In [12]:
config = tf.ConfigProto(log_device_placement = True, allow_soft_placement=True)
config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess :
    coord = tf.train.Coordinator()
    thread = tf.train.start_queue_runners(sess=sess, coord=coord)
        
    sess.run(tf.global_variables_initializer())
    
    total_batch = int(len(image_list) / batch_size)
    correct_list = []
    incorrect_list = []
    cost_list = []
    cost_sum_list = []
    pred_list = []
    total_acc = 0
    cost_sum = 0
    cost_flag = 0
    rate_ = 1
    
    start_time = time.time()
    
    print("")
    
    for epoch in range(training_epochs) :
        total_cost = 0
        
        if epoch%50 == 49 :
                rate_ *= 0.8
                
        if epoch%10 == 9 :
#             rate_ *= 0.8
            midpoint = int(time.time() - start_time)
        
            print("")
            print("=======================================================================================")
            print('{:03d}:{:02d}:{:02d}'.format(midpoint//3600, (midpoint%3600//60), midpoint%60))
            print("=======================================================================================")
            print("")
        
        batch_index = np.random.choice(len(image_array), total_batch, replace=False)
                
        for i in range(total_batch) :                
            _, _loss = sess.run([train, loss], feed_dict={x: image_array[[batch_index[i]]], y_: label_array[[batch_index[i]]], 
                                                          is_training: True, learning_rate: initial_rate*rate_,
                                                          W_h0_1: weight0_1, W_h0_2: weight0_2,
                                                          W_h1: weight1, W_h1_1: weight1_1, W_h1_2: weight1_2,
                                                          W_h2: weight2, W_h2_1: weight2_1, W_h2_2: weight2_2,
                                                          W_h3: weight3, W_h3_1: weight3_1, W_h3_2: weight3_2})                 
            total_cost += _loss

        avg_cost = total_cost/total_batch
        print('Epoch : ', '%4d' % (epoch + 1), '    Avg. cost = ', '{:.4f}'.format(avg_cost))            
        
#         train_writer.add_summary(_merge, epoch)
               
    print("")
    print("=======================================================================================")
    print("================================     Training done     ================================")
    print("=======================================================================================")
    print("")
    
    for test in range(len(test_image_array)) :
        _acc, _label_input, _label_output, _pred = sess.run([acc, label_input, label_output, pred], 
                             feed_dict={x: test_image_array[[test]], y_: test_label_array[[test]], is_training: False,
                                        W_h0_1: weight0_1, W_h0_2: weight0_2,
                                        W_h1: weight1, W_h1_1: weight1_1, W_h1_2: weight1_2,
                                        W_h2: weight2, W_h2_1: weight2_1, W_h2_2: weight2_2,
                                        W_h3: weight3, W_h3_1: weight3_1, W_h3_2: weight3_2})

        total_acc += _acc
        pred_list.append(list(_pred[0]))
        
        print("n: ", test, "    label_input: ", _label_input, "    label_output: ", _label_output)
        
        if _label_input == _label_output :
            correct_list.append(test)   
        else :
            incorrect_list.append(test)
        
    print("")
    print("          TOTAL ACC    : ", '{:.5f}'.format(total_acc / len(test_image_array)))
    print("")
    print(pred_list)
    
    end_time = int(time.time() - start_time)
    
    print("")
    print("=======================================================================================")
    print('{:03d}:{:02d}:{:02d}'.format(end_time//3600, (end_time%3600//60), end_time%60))
    print("=======================================================================================")
    print("")
    
    coord.request_stop()
    coord.join(thread)

InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: the launch timed out and was terminated